# Test code for computing registration metrics (reproducibilty) 

1. Check xcorr_vol between two outputs of the two preproc runs 
2. Check registration params
    - xfminvert stx2_052_S_1251_20070309_t1.xfm test.xfm
    - xfm2param test.xfm


**Expected use case:** 
    Run this script after preprocessing is complete to provide info on each subject

In [3]:
import pandas as pd
import numpy as np
import sys
import os
import re
import subprocess
sys.path.append('../')
from lib.preproc_checks import *

# Data paths

In [29]:
proj_dir = '/Users/nikhil/projects/MR_preproc_dash/'
data_dir = proj_dir + 'data/'
script_dir = proj_dir + 'scripts/'
preproc_pipeline_dir = '/Users/nikhil/code/git_repos/nist_mni_pipelines/'

if preproc_pipeline_dir not in sys.path:
    sys.path.append(preproc_pipeline_dir)

# Calling minc from python
**Using subprocess (a bit hacky but easier than pyminc interface!)**

In [91]:
vol1 = data_dir + '052_S_4807/20121019/stx/stx_052_S_4807_20121019_t1.mnc'
vol2 = data_dir + '052_S_4807/20121019/stx/stx_052_S_4807_20121019_t1.mnc'
xfm1 = data_dir + '052_S_4807/20121019/stx/stx_052_S_4807_20121019_t1.xfm'

xcorr = get_xcorr_vol(vol1,vol2)
reg_param = get_reg_params(xfm1)
print(xcorr)
print(reg_param)

1.0
                   x        y        z
center             0        0        0
translation   2.0631  43.5027  7.50818
rotation     2.59585  0.26141 -0.98333
scale        0.94557  0.95997  0.87936
shear        0.00055 -0.00056  0.00873


In [88]:
def get_xcorr_vol(vol1,vol2):
    minc_xcorr_cmd = script_dir + 'run_xcorr_cmd.sh' + ' ' + vol1 + ' ' + vol2
    xcorr = 0
    try:
        xcorr = float(subprocess.check_output(minc_xcorr_cmd, shell=True))
    except:
        print('Could not run minc command. Check minc script / command: \n{}\n'.format(minc_xcorr_cmd))
        
    return xcorr

def get_reg_params(xfm):
    minc_reg_param_cmd = script_dir + 'run_reg_param_cmd.sh' + ' ' + xfm1
    reg_df = pd.DataFrame(index=['center','translation','rotation','scale','shear'],columns=['x','y','z'])
    string_check = False
    origin_check = False
    
    try: 
        reg_param_str = str(subprocess.check_output(minc_reg_param_cmd, shell=True),'utf-8')
        #print(reg_param_str)
        reg_param_split = str.split(reg_param_str,' ')        

        # Check if output string is what you expect
        string_check = ((reg_param_split[3]=='-center') & (reg_param_split[7]=='-translation') & 
                        (reg_param_split[11]=='-rotation') & (reg_param_split[15]=='-scale') & 
                        (reg_param_split[19]=='-shear'))

        # Check if origin = [0,0,0]
        if string_check:
            origin_check = (np.array(reg_param_split[4:7]).astype(float) == [0,0,0]).all()

            if origin_check:
                reg_df.loc['center'] = np.array(reg_param_split[4:7]).astype(float)
                reg_df.loc['translation'] = np.array(reg_param_split[8:11]).astype(float)
                reg_df.loc['rotation'] = np.array(reg_param_split[12:15]).astype(float)
                reg_df.loc['scale'] = np.array(reg_param_split[16:19]).astype(float)
                reg_df.loc['shear'] = np.array(reg_param_split[20:23]).astype(float)
            else:
                print('Origin is not at [0,0,0]. Check input image space.')
        else: 
            print('minc command output string doesnot match expected output from xfm2param')
    except:
        print('Could not run minc command. Check minc script / command: \n{}\n'.format(minc_reg_param_cmd))

    return reg_df
